## Punto 2 y 3. Cálculo de las toxicidades genéricas específicas para cada modelo de tejido sano; y cálculo de los targets genéticos para cada modelo de mieloma múltiple.

In [1]:
import cplex
import cobra
import time
import pickle
import sys
cobra.Configuration().solver='cplex'

In [ ]:
start=time.time()

In [2]:
path="../Entrega Paco/"
nombreInicial=sys.argv[1]
nombreFinal=sys.argv[2]
with open(path+nombreInicial, "rb") as f:
    modelo=pickle.load(f)

In [3]:
susGenes=dict() #Vamos a guardar los genes clave de cada reaccion en un diccionario
for r in modelo.reactions:
    grr=r.gene_reaction_rule
    if len(r.genes)==0: #Si no tiene genes, pues declaramos como un set vacío
        susGenes[r.id]=set()
    if len(r.genes)==1: #Si solo tiene un gen, lo guardamos, lo queremos
        susGenes[r.id]=set([g.id for g in r.genes]) #Nos quedamos con el gen cuando solo tiene 1
    if "or" in grr and not "and" in grr: #si tiene OR pero no AND, pasamos de los genes
        susGenes[r.id]=set()
    if "and" in grr and not "or" in grr: #Si solo tenemos "AND", separamos los genes y los guardamos todos
        susGenes[r.id]=set(grr.split(" and ")) #Ponemos " and " para que nos separe y nos de una lista con los genes, sin espacios
    if "and" in grr and "or" in grr: # Si tiene "AND" y "OR" tambie nos quedamos todos los genes
        susGenes[r.id]=set([g.id for g in r.genes])

7904


In [4]:
biomasa=modelo.reactions.MAR13082 
# Buscamos el soporte
error=10**-12
with modelo:
    # El objetivo de este modelo por defecto es la BIOMASA, entonces no hace falta indicarlo
    modelo.objective=biomasa
    modelo.objective_direction="max"
    sol=modelo.optimize() #No hago el SLIM, porque luego quiero ver que reacciones están activas en la solución
    sop=[r.id for i,r in enumerate(modelo.reactions) if abs(sol.fluxes[i])>error]
    #Guardamos las reacciones cuyo flujo es positivo

1054


In [5]:
# Ahora vemos que genes podemos cargarnos para cargarnos cada reacción
# del soporte obtenido

candidatos=set()
for r in sop:
    candidatos.update(susGenes[r])

352


In [6]:
maxBiomasa=modelo.slim_optimize()

In [7]:
gMCSs=set()
for g in candidatos:
    with modelo:
        modelo.genes.get_by_id(g).knock_out()
        biomasa.bounds=[maxBiomasa/1000,10000]
        sol=modelo.slim_optimize(error_value=-1)
        if sol==-1:
            gMCSs.add(g)

122

In [ ]:
with open(path+nombreFinal, "wb") as f:
    pickle.dump(gMCSs,f)

In [ ]:
print(len(gMCSs), time.time()-start)